In [1]:
import numpy as np

In [2]:
def make_grid(user_input):
    """Transforms user input into a two-dimensional Numpy array"""
    grid = np.reshape(user_input, (9, 9))
    return grid

In [3]:
def check_empty_cell(grid, position):
    """Determines the next empty cell to be filled with a number by the Sudoku solver"""
    value = grid[position // 9][position % 9]
    return not value

In [4]:
def found_in_row(grid, position, num):
    """Finds whether the suggested number is already in the row"""
    return num in grid[position // 9]

In [5]:
def found_in_col(grid, position, num):
    """Finds whether the suggested number is already in the column"""
    return num in grid[:, position % 9]

In [6]:
def found_in_block(grid, position, num):
    """Finds whether the suggested number is already in the same 3x3 block"""
    return (
        num
        in grid[
            3 * ((position // 9) // 3) : (3 * ((position // 9) // 3) + 3),
            3 * ((position % 9) // 3) : (3 * ((position % 9) // 3) + 3),
        ]
    )

In [7]:
def check_safe_cell(grid, position, num):
    """Checks whether the suggested number can safely be placed in the cell"""
    return (
        not found_in_row(grid, position, num)
        and not found_in_col(grid, position, num)
        and not found_in_block(grid, position, num)
    )

In [8]:
def solve_sudoku(grid):
    """Implements a backtracking algorithm. 
    
    We try filling empty cells one by one. 
    
    Whenever we find that the current cell cannot lead to a viable solution, we empty it and we backtrack to the previous cell.
    
    The function takes as input a grid from the user and returns the solved sudoku.
    
    """

    # Keep initial grid to recall which cells are empty
    initial_grid = np.array(grid, copy=True)

    # Initialize variables
    onwards = True
    position = 0

    # The algorithm solves the grid row by row, from left to right.
    while position < 81:

        # If cell is empty and we are not backtracking
        if check_empty_cell(initial_grid, position) and onwards:
            for num in range(1, 10):

                # Try to put the first safe number starting at 1
                if check_safe_cell(grid, position, num):
                    grid[position // 9, position % 9] = num
                    position += 1
                    break

                # If no number fits (1-9), there is a mistake in a preceding cell and we have to backtrack
                else:
                    if num == 9:
                        onwards = False
                        position -= 1
                        break

        # If cell is not empty and we are not backtracking (going onwards): go to next cell
        elif not check_empty_cell(initial_grid, position) and onwards:
            position += 1

        # If cell is empty but we are backtracking
        elif check_empty_cell(initial_grid, position) and not onwards:

            # If number already at 9, empty cell and go 1 cell backwards
            if grid[position // 9, position % 9] == 9:
                grid[position // 9, position % 9] = 0
                position -= 1

            # If number less than 9: try next number(s)
            else:
                for num in range(grid[position // 9, position % 9] + 1, 10):
                    if check_safe_cell(grid, position, num):
                        grid[position // 9, position % 9] = num
                        onwards = True
                        position += 1
                        break
                    else:
                        if num == 9:
                            grid[position // 9, position % 9] = 0
                            position -= 1
                            break

        # If cell filled in the initial grid and we are backtracking, go 1 cell backwards
        elif not check_empty_cell(initial_grid, position) and not onwards:
            position -= 1

    return grid